In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import logging 
import os
import time
from gtts import gTTS

#TODO: Remove Automod comments

logger = logging.getLogger(__name__)

BASE_OUTPUT_DIR = "output"
COMMENT_OUTPUT_DIR = os.path.join(BASE_OUTPUT_DIR, "comments")
POST_DIR = os.path.join(BASE_OUTPUT_DIR, "post")
POST_AUDIO_DIR = os.path.join(POST_DIR, "audio")
POST_IMAGE_DIR = os.path.join(POST_DIR, "images")
COMMENT_AUDIO_DIR = os.path.join(COMMENT_OUTPUT_DIR, "audio")
COMMENT_IMAGE_DIR = os.path.join(COMMENT_OUTPUT_DIR, "images")
DIRECTORIES = [POST_AUDIO_DIR, POST_IMAGE_DIR, COMMENT_AUDIO_DIR, COMMENT_IMAGE_DIR]

link = "https://www.reddit.com/r/MachineLearning/comments/11rtzv6/d_what_do_people_think_about_openai_not_releasing/"

def make_dir_if_not_exists(dir_name):
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)

def delete_content_of_dir(dir_name):
    if os.path.exists(dir_name):
        for file in os.listdir(dir_name):
            os.remove(os.path.join(dir_name, file))


class Viddit:
    def __init__(self, path_to_driver = "chromedriver.exe"):
        options = Options()
        # options.add_argument('--headless')
        # options.add_argument('--disable-gpu')  # Last I checked this was necessary.
        self.driver = webdriver.Chrome(path_to_driver, chrome_options=options)
        self.setup()

    def setup(self):
        [make_dir_if_not_exists(directory) for directory in DIRECTORIES]

    def delete_data(self):
        [delete_content_of_dir(directory) for directory in DIRECTORIES]
    
    def teardown(self):
        self.delete_data()
        self.driver.close()
        self.driver.quit()

    def accept_cookies(self):
        self.driver.get("https://www.reddit.com/")
        WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.XPATH, "//button[contains(text(), 'Accept all')]")))
        self.driver.find_element("xpath", "//button[contains(text(), 'Accept all')]").click()
        logger.info("Cookies accepted")
        self.driver.switch_to.default_content() 

    def scrape(self, post_url):
        # Load cookies to prevent cookie overlay & other issues
        # for cookie in config['reddit_cookies'].split('; '):
        #     cookie_data = cookie.split('=')
        #     driver.add_cookie({'name':cookie_data[0],'value':cookie_data[1],'domain':'reddit.com'})

        # Fetching the post itself, text & screenshot
        self.driver.get(post_url)
        post = WebDriverWait(self.driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, '.Post')))
        post_text = post.find_element(By.CSS_SELECTOR, 'h1').text #TODO Text to speech
        post.screenshot(os.path.join(POST_IMAGE_DIR, "0.png"))
        tts = gTTS(post_text)
        tts.save(os.path.join(POST_AUDIO_DIR, "0.mp3"))
        # Let comments load
        self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3) #TODO Can be a WebDriverWait
        
        # Fetching comments & top level comment determinator
        comments = WebDriverWait(self.driver, 20).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div[id^=t1_][tabindex]')))
        allowed_style = comments[0].get_attribute("style")
        
        # Filter for top only comments
        NUMBER_OF_COMMENTS = 10
        comments = [comment for comment in comments if comment.get_attribute("style") == allowed_style][:NUMBER_OF_COMMENTS]

        logger.info('💬 Scraping comments...')
        # Save time & resources by only fetching X content
        for i in range(len(comments)):
            # TODO Filter out locked comments (AutoMod) 
            # Scrolling to the comment ensures that the profile picture loads
            # Credits: https://stackoverflow.com/a/57630350
            desired_y = (comments[i].size['height'] / 2) + comments[i].location['y']
            window_h = self.driver.execute_script('return window.innerHeight')
            window_y = self.driver.execute_script('return window.pageYOffset')
            current_y = (window_h / 2) + window_y
            scroll_y_by = desired_y - current_y
            self.driver.execute_script("window.scrollBy(0, arguments[0]);", scroll_y_by)
            time.sleep(0.2)

            # Getting comment into string TODO TTS
            text = "\n".join([element.text for element in comments[i].find_elements(By.CSS_SELECTOR,'.RichTextJSON-root')])
            tts = gTTS(text)
            tts.save(os.path.join(COMMENT_AUDIO_DIR, f'{i}.mp3'))
            # Screenshot & save text
            comments[i].screenshot(os.path.join(COMMENT_IMAGE_DIR, f'{i}.png'))
        return self.driver
    
viddit = Viddit()
viddit.accept_cookies()
viddit.scrape(link)

C:\Users\JamMa\AppData\Local\Temp\ipykernel_33248\3880542425.py:40: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = webdriver.Chrome(path_to_driver, chrome_options=options)
C:\Users\JamMa\AppData\Local\Temp\ipykernel_33248\3880542425.py:40: DeprecationWarning: use options instead of chrome_options
  self.driver = webdriver.Chrome(path_to_driver, chrome_options=options)


AssertionError: No text to speak

In [4]:
viddit.delete_data()

In [ ]:
import numpy as np
import soundfile as sf
import yaml
import tensorflow as tf
from tensorflow_tts.inference import TFAutoModel
from tensorflow_tts.inference import AutoProcessor

# initialize fastspeech2 model.
fastspeech2 = TFAutoModel.from_pretrained("tensorspeech/tts-fastspeech2-ljspeech-en")
# initialize mb_melgan model
mb_melgan = TFAutoModel.from_pretrained("tensorspeech/tts-mb_melgan-ljspeech-en")
# inference
processor = AutoProcessor.from_pretrained("tensorspeech/tts-fastspeech2-ljspeech-en")
input_ids = processor.text_to_sequence("Hello from a computer.")
# fastspeech inference
mel_before, mel_after, duration_outputs, _, _ = fastspeech2.inference(
    input_ids=tf.expand_dims(tf.convert_to_tensor(input_ids, dtype=tf.int32), 0),
    speaker_ids=tf.convert_to_tensor([0], dtype=tf.int32),
    speed_ratios=tf.convert_to_tensor([1.0], dtype=tf.float32),
    f0_ratios =tf.convert_to_tensor([1.0], dtype=tf.float32),
    energy_ratios =tf.convert_to_tensor([1.0], dtype=tf.float32),
)

# melgan inference
audio_before = mb_melgan.inference(mel_before)[0, :, 0]
audio_after = mb_melgan.inference(mel_after)[0, :, 0]

# save to file
sf.write('./audio_before.wav', audio_before, 22050, "PCM_16")
sf.write('./audio_after.wav', audio_after, 22050, "PCM_16")

In [ ]:
import cv2
import numpy as np
from moviepy.editor import VideoFileClip, AudioFileClip, CompositeAudioClip, concatenate_videoclips

def overlay_images_on_video(background_video_path, png_paths, audio_paths, audio_volume, output_path):
    # Open the video file
    cap = cv2.VideoCapture(background_video_path)

    # Get the video properties
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    logger.debug(f'Video properties: {fps} fps, {width}x{height} pixels')


    # Create a list of video clips for each PNG
    clips = []
    for i in range(len(png_paths)):
        # Read the PNG image
        png = cv2.imread(png_paths[i], cv2.IMREAD_UNCHANGED)
    # Calculate the total number of frames for the display time of each PNG

    # Open the audio file and set its volume
        audio_clip = AudioFileClip(audio_paths[i]).volumex(audio_volume)
        num_frames = int(audio_clip.duration * fps) + fps
        # Calculate the size and position of the PNG image to be centered on the video
        png_height, png_width, _ = png.shape
        x = int((width - png_width) / 2)
        y = int((height - png_height) / 2)

        # Create a mask for the PNG image
        alpha_channel = png[:, :, 3]
        mask = np.zeros((png_height, png_width), dtype=np.uint8)
        mask[alpha_channel > 0] = 255

        # Create a list of video frames for the current PNG
        frames = []
        for i in range(num_frames):
            ret, frame = cap.read()
            if ret:
                # Resize the PNG image to match the size of the video frame
                png_resized = cv2.resize(png[:, :, :3], (frame.shape[1], frame.shape[0]))

                # Apply the mask to the PNG image
                mask_resized = cv2.resize(mask, (frame.shape[1], frame.shape[0]))
                mask_resized = cv2.cvtColor(mask_resized, cv2.COLOR_GRAY2BGR)
                png_masked = cv2.bitwise_and(png_resized, mask_resized)

                # Overlay the PNG image on the video frame
                frame[y:y+png_height, x:x+png_width] = cv2.add(frame[y:y+png_height, x:x+png_width], png_masked)

                # Add the frame to the list of frames
                frames.append(frame)
            else:
                break

        # Create a video clip from the list of frames and add it to the list of clips
        clip = VideoFileClip(None, fps=fps).set_duration(num_frames/fps)
        clip = clip.set_position('center')
        clip = clip.set_make_frame(lambda t: frames[int(t * fps)])
        clip.set_audio(audio_clip)
        clips.append(clip)

    # Concatenate the list of video clips and set the audio
    video_clip = concatenate_videoclips(clips)
    video_clip = video_clip.set_audio(CompositeAudioClip([audio_clip for _ in range(len(clips))]))
    video_clip.write_videofile(output_path, fps=fps)
    video_clip.close()